## Heap과 Priority Queue

### 1. 우선순위 큐 (priority queue)
> 들어간 순서와 관계없이 우선순위가 높은 요소가 먼저 나오는 자료구조

### 2. 힙 (Heap)
> 완전 이진 트리 기반의 자료구조로, 부모-자식 간 대소 관계가 정해진 구조

- 최소 힙
  - 부모 노드 <= 자식 노드
  - 루트가 최솟값
- 최대 힙
  - 부모 노드 >= 자식 노드

### 핵심연산 시간 복잡도
| 연산 | 시간 복잡도 | 설명 |
|------|------------|------|
| heappush | O(log n) | 삽입 후 상향 조정 |
| heappop | O(log n) | 삭제 후 하향 조정 |
| heap[0] | O(1) | 최솟값 확인 |
| heapify | O(n) | 리스트를 힙으로 변환 |
| heappushpop | O(log n) | push 후 pop (최적화) |
| heapreplace | O(log n) | pop 후 push (최적화) |


### 우선순위 큐와 힙
- 힙
  - 자료구조 그 자체 
  - 먼저 처리할 것부터 처리한다는 규칙을 효율적으로 구현하는 도구
- 우선순위 큐
  - 추상적인 개념, 우선순위에 따라 요소를 꺼낸다 는 동작만 정의
  - 먼저 처리할 것부터 처리한다 는 규칙
- 대부분의 경우 우선순위 큐를 힙으로 봐도 된다
  - 힙이 우선순위 큐의 가장 효율적인 구현이기 때문


In [2]:
# 일반 큐
from collections import deque

queue = deque()
queue.append(5)
queue.append(2)
queue.popleft()

5

In [3]:
# 힙 (우선순위 큐)
import heapq

pq = []
heapq.heappush(pq, 5)
heapq.heappush(pq, 2)
heapq.heappop(pq)

2

---

### 215. Kth Largest Element in an Array

Given an integer array nums and an integer k, return the kth largest element in the array.

Note that it is the kth largest element in the sorted order, not the kth distinct element.

Can you solve it without sorting?



In [16]:
from typing import List
import heapq

class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        nums = [-i for i in nums]
        heapq.heapify(nums)

        for _ in range(k):
            result = heapq.heappop(nums)

        return -result

---

### 2336. Smallest Number in Infinite Set
You have a set which contains all positive integers [1, 2, 3, 4, 5, ...].

Implement the SmallestInfiniteSet class:

- SmallestInfiniteSet() Initializes the SmallestInfiniteSet object to contain all positive integers.
- int popSmallest() Removes and returns the smallest integer contained in the infinite set.
- void addBack(int num) Adds a positive integer num back into the infinite set, if it is not already in the infinite set.


In [ ]:
import heapq

class SmallestInfiniteSet:

    def __init__(self):
        self.current = 1
        self.heap = []
        self.remove = set()

    def popSmallest(self) -> int:
        if self.heap and self.heap[0] < self.current:
            val = heapq.heappop(self.heap)
            self.remove.remove(val)
            return val
        else:
            val = self.current
            self.current += 1
            return val


    def addBack(self, num: int) -> None:
        if num < self.current and num not in self.remove:
            heapq.heappush(self.heap, num)
            self.remove.add(num)


# Your SmallestInfiniteSet object will be instantiated and called as such:
# obj = SmallestInfiniteSet()
# param_1 = obj.popSmallest()
# obj.addBack(num)

---

### 2542. Maximum Subsequence Score
You are given two 0-indexed integer arrays nums1 and nums2 of equal length n and a positive integer k. You must choose a subsequence of indices from nums1 of length k.

For chosen indices i0, i1, ..., ik - 1, your score is defined as:

- The sum of the selected elements from nums1 multiplied with the minimum of the selected elements from nums2.
- It can defined simply as: (nums1[i0] + nums1[i1] +...+ nums1[ik - 1]) * min(nums2[i0] , nums2[i1], ... ,nums2[ik - 1]).

Return the maximum possible score.

A subsequence of indices of an array is a set that can be derived from the set {0, 1, ..., n-1} by deleting some or no elements.



In [20]:
class Solution:
    def maxScore(self, nums1: List[int], nums2: List[int], k: int) -> int:
        
        paris = sorted(zip(nums2, nums1), reverse=True)

        minheap = []
        sum_num1 = 0
        answer = 0

        for n2, n1 in paris:
            heapq.heappush(minheap, n1)
            sum_num1 += n1

            if len(minheap) > k:
                sum_num1 -= heapq.heappop(minheap)
            
            if len(minheap) == k:
                answer = max(answer, sum_num1 * n2)

        return answer

s = Solution()
s.maxScore(nums1 = [1,3,3,2], nums2 = [2,1,3,4], k = 3)

12

---

### 2462. Total Cost to Hire K Workers
You are given a 0-indexed integer array costs where costs[i] is the cost of hiring the ith worker.

You are also given two integers k and candidates. We want to hire exactly k workers according to the following rules:

- You will run k sessions and hire exactly one worker in each session.
- In each hiring session, choose the worker with the lowest cost from either the first candidates workers or the last candidates workers. Break the tie by the smallest index.
    - For example, if costs = [3,2,7,7,1,2] and candidates = 2, then in the first hiring session, we will choose the 4th worker because they have the lowest cost [3,2,7,7,1,2].
    - In the second hiring session, we will choose 1st worker because they have the same lowest cost as 4th worker but they have the smallest index [3,2,7,7,2]. Please note that the indexing may be changed in the process.
- If there are fewer than candidates workers remaining, choose the worker with the lowest cost among them. Break the tie by the smallest index.
- A worker can only be chosen once.

Return the total cost to hire exactly k workers.

In [ ]:
class Solution:
    def totalCost(self, costs: List[int], k: int, candidates: int) -> int:
        left_heap = []
        right_heap = []

        l, r = 0, len(costs) - 1

        for _ in range(candidates):
            if l > r: break
            heapq.heappush(left_heap, costs[l])
            l += 1
        
        for _ in range(candidates):
            if l > r: break
            heapq.heappush(right_heap, costs[r])
            r -= 1

        answer = 0

        for _ in range(k):

            # 둘다 차있는 경우
            if left_heap and right_heap:
                if left_heap[0] <= right_heap[0]:
                    val = heapq.heappop(left_heap)
                    answer += val
                    if l <= r:
                        heapq.heappush(left_heap, costs[l])
                        l += 1

                else:
                    val = heapq.heappop(right_heap)
                    answer += val
                    if l <= r:
                        heapq.heappush(right_heap, costs[r])
                        r -= 1

            # 왼쪽만 남은 경우
            elif left_heap:
                val = heapq.heappop(left_heap)
                answer += val
                if l <= r:
                    heapq.heappush(left_heap, costs[l])
                    l += 1

            else:
                val = heapq.heappop(right_heap)
                answer += val
                if l <= r:
                    heapq.heappush(right_heap, costs[r])
                    r -= 1

        return answer